# Update DOIs

In [1]:
import urllib
import urllib.request
import urllib.parse
import urllib.error
import requests
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
from pathlib import Path
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from functools import reduce
from start import data_path
import extract_laws
import clean_documents
import numpy as np
import gather_documents

In [2]:
# import cleaned exemptions list
docs_df = pd.read_csv(os.path.join(data_path,'doi_exemptions_list.csv'))
print(len(docs_df))
docs_df[docs_df.title == "South San Antonio ISD"]

824


,title,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,p_innovation,possible_laws
693,South San Antonio ISD,130.0,614,614,614,640,Second,pdf,https://www.southsanisd.net//cms/lib/TX0191831...,South San Antonio Independent School District ...,0.999955,"[25.0811, 21.003, 25.112, 25.1113, 21.051, 25...."


In [3]:
# old districts with missing laws
missing_laws = docs_df[docs_df.possible_laws == '[]']
len(missing_laws)
missing_laws_list = list(missing_laws.title)
missing_laws_list

['Bronte\xa0ISD',
 'Burleson ISD',
 'Crosbyton CISD',
 'Dimmitt ISD',
 'Gregory-Portland ISD',
 'Henrietta ISD',
 'La Joya ISD',
 'Liberty Hill ISD',
 'Marathon ISD',
 'Midway ISD (161903) ']

In [4]:
# import cleaned dates

## Check TEA website for new districts of innovation

In [5]:
url = "https://tea.texas.gov/Texas_Schools/District_Initiatives/Districts_of_Innovation/"
webcontent = urllib.request.urlopen(url).read()
soup = BeautifulSoup(webcontent, 'html.parser')
links = soup.find_all('a')
districts_list = [i for i in map(lambda x: x.get('title'), links) 
                  if i is not None and 'ISD' in i]
print("Number of districts on DOI website", len(districts_list))

Number of districts on DOI website 896


In [14]:
new_districts_list = []
for dist in districts_list:
    if dist not in list(docs_df.title):
        new_districts_list.append(dist)

print("Number of new districts: ", len(new_districts_list))

Number of new districts:  76


In [15]:
new_districts_list = new_districts_list + missing_laws_list
new_df = pd.DataFrame(new_districts_list, columns=['title'])
print(len(new_df))

86


In [16]:
# Gather

In [17]:
first_level_links = gather_documents.FirstLevelLinks(url, print_interim=False)
first_level_df = first_level_links.docs_df.reset_index().rename(columns={'index': 'title'})
print(len(first_level_df))
first_level_df

450


,title,link,type
0,Denver City ISD,http://www.dcisd.org/cms/lib011/TX01917797/Cen...,docx
1,Marion ISD,http://www.marionisd.net/upload/page/0020/DofI...,docx
2,Post ISD,https://1.cdn.edl.io/I2a9qAWX4QHUGdG5HRdQP38Ja...,docx
3,Abilene ISD,https://www.abileneisd.org/wp-content/uploads/...,pdf
4,Adrian ISD,https://s3.amazonaws.com/scschoolfiles/2280/ad...,pdf
...,...,...,...
445,Valentine ISD,https://drive.google.com/file/d/1xmFIMZZfvcs4u...,google
446,Vega ISD,https://drive.google.com/file/d/1ecypqBHsR1b4-...,google
447,Whitehouse ISD,https://docs.google.com/document/d/1lxtD2uHrmK...,google
448,Whitharral ISD,https://docs.google.com/document/d/1QpBqSAmJNZ...,google


In [20]:
new_df = new_df.merge(first_level_df, how = 'left')
print(len(new_df))
new_df
new_df.to_csv(os.path.join(data_path, 'update_links.csv'))

86


In [31]:
# Only keep new districts
new_districts_links_df = first_level_df[first_level_df['title'].isin(new_districts_list)]
len(new_districts_links_df)
len(new_districts_links_df)

43